# State Value Function

Next we will find different ways to compute the Value function given by a stochastic policy $\pi(s) = p(a\mid s)$.
We want to calculate $V_{\pi}(s)$ (the state-value-function given a policy)
Here, we draw an Markov Decision Process (MDP) with three states $\mathcal{S}=\{s_1,s_2,s_3\}$ and three possible actions $\mathcal{A}=\{a_1,a_2,a_3\}$, moving to state $s_1$, moving to state $s_2$ and moving to state $s_3$.
![mdp.png](mdp.png)

# 6.1 Policy Evaluation by Dynamic Programming

For the MDP represented above we define the state transition probability matrix $\mathcal{P}^a_{ss'}=p(S_{t+1}=s'\mid S_{t}=s, A_t=a)$. In this MDP we assume that when we choose to move to state $s_i$, $i=\{1,2,3\}$ we always end up in that state, meaning that $\mathcal{P}^a_{ss'}=p(S_{t+1}=s'\mid S_{t}=s, A_t=a)=1$. In this case, $\mathcal{P}^{\pi}=\mathcal{P}^a_{ss'}\pi(a\mid s) = \pi(a\mid s)$ the Bellman Expectation equation becomes (Check page 14 and 16 from the lecture slides.):
$$
V_{\pi}(s) = \sum_{a\in\mathcal{A}} \pi(a\mid s)\left( \mathcal{R}^a_s + \gamma \sum_{s'\in \mathcal{S}}\mathcal{P}^a_{ss'}V_{\pi}(s')\right) = \mathcal{R}^{\pi}+ \gamma \sum_{s'\in \mathcal{S}}\pi(a\mid s)V_{\pi}(s')
$$

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

policy=np.array([[0.3, 0.2, 0.5], [0.5, 0.4, 0.1], [0.8, 0.1, 0.1]])
# 'raw_rewards' variable contains rewards obtained after transition to each state
# In our example it doesn't depend on source state
raw_rewards = np.array([1.5, -1.833333333, 19.833333333])
# 'rewards' variable contains expected values of the next reward for each state
rewards = np.matmul(policy, raw_rewards)
assert np.allclose(rewards, np.array([10., 2., 3.]))

state_value_function=np.array([0 for i in range(3)])

for i in range(20):
    print(state_value_function)
    
    state_value_function=#TODO: Implement the Policy Evaluation Update with a Discount Rate of 0.1
print(state_value_function)

# 6.1 Policy Evaluation by Linear Programming
The state-value-function can be directly solved through linear programming (as shown on page 15 from the lecture slides):
$$
V_{\pi}(s)=\left(I-\gamma\mathcal{P}^{\pi}\right)^{-1}\mathcal{R}^{\pi}
$$

In [ ]:
solution=#TODO: Implement the linear programming solution with a discount rate of 0.1
print(solution)

The result should stay the same.

# 6.2 Monte Carlo Policy Evaluation

We can design yet another way of evaluating the value of a given policy $\pi$, see lecture slides pag.20.
The intuition is to incrementally the expected return from sampled episodes, sequences of triplets $\{(s_i,a_i,r_{i})\}_{i=1}^N$. The function $\color{blue}{gt}$ computes the total discounted reward from a list of sequential rewards obtained by sampling the policy over N time-steps: $G_t=r_t+\gamma r_{t+1}+\gamma^2 r_{t+2}+\dots+\gamma^N r_{t+N}$.

The value of a policy can also be computed by looking at its empirical expected total discounted reward:
$$
V_{\pi}(s) = \mathbb{E}_{\pi}\left[G_t\mid S_t=s\right]
$$

In [ ]:
import random
from collections import defaultdict
reward_counter=np.array([0., 0., 0.])
visit_counter=np.array([0., 0., 0.])

def gt(rewardlist, gamma=0.1):
    '''
    Function to calculate the total discounted reward
    >>> gt([10, 2, 3], gamma=0.1)
    10.23
    '''
    #TODO: Implement the total discounted reward
    return 0


for i in range(400):
    start_state=random.randint(0, 2)
    next_state=start_state
    rewardlist=[]
    occurence=defaultdict(list) 
    for i in range(250):
        rewardlist.append(rewards[next_state]) 
        occurence[next_state].append(len(rewardlist)-1) 
        action=np.random.choice(np.arange(0, 3), p=policy[next_state]) 
        next_state=action

    for state in occurence: 
        for value in occurence[state]: 
            rew=gt(rewardlist[value:]) 
            reward_counter[state]+=rew 
            visit_counter[state]+=1 
            #break #if break: return following only the first visit

print(reward_counter/visit_counter)

As can be seen the result is nearly the same as the state-value-function calculated above.

So far we have seen different ways of given a known policy $\pi(a\mid s)$ how to comput its value $V_{\pi}(s)$. Next, we wish to find the optimal policy $\pi^\ast(s)$ for the MDP in the example.

# 6.3 Policy Optimization by Q-Learning

This code solves a very easy problem: using the rewards it calculates the optimal action-value-function (page 26 on slides).

It samples a state-action pair randomly, so that all state-action pairs can be seen, and updates the matrix of Q-values (expected discounted reward under the policy), for each state-action pair $\mathbf{Q[s_t, a_t] = (1-\alpha)\ Q[s_t, a_t] + \alpha \left( r_t + \gamma \max\limits_{a} Q[s_{t+1}, a]\right)}$ since we are in the tabular case--- where states and action are finite.

In [ ]:
q_table=np.zeros((3, 3)) 
for i in range(1001): 
    state=random.randint(0, 2) 
    action=random.randint(0, 2) 
    next_state=action
    reward=rewards[next_state] 
    next_q=max(q_table[next_state]) 
    q_table[state, action]= #TODO: Implement the Q-Table update
    if i%100==0:
        print(q_table)

# Extra: Value Iteration

Next we compute the optimal policy by first optimizing the value function, via a fixed point method, using the Bellman optimality equation for the state value function: $ V(s) = \max_{a\in\mathcal{A}}r_t + \gamma \sum_{s'\in \mathcal{S}}\mathcal{P}^a_{s_t,s'}V(s')$, where for the given MDP $\mathcal{P}^a_{ss'}=p(S_{t+1}=s'\mid S_{t}=s, A_t=a)=1$.

In [ ]:
import numpy as np

rewards=np.array([10., 2., 3.])
gamma = 0.1

state_value_function = np.zeros(3)

for i in range(1000):
    for s in range(3):
        state_value_function[s]=#TODO: Implement the state value function update
print(state_value_function)

# Exercise 6.4: Score Function Gradient Estimator

Implement the score function gradient estimator of the REINFORCE algorithm in lxmls/reinforcement_learning/score\_function\_estimator.py. Check it is correct by calling the train() function.

In [ ]:
%matplotlib inline
from lxmls.reinforcement_learning.score_function_estimator import train
train()

# Exercise 6.5: Policy Gradient for the CartPole task

Implement policy gradient for the cartpole task by coding the forward pass of Model() in lxmls/reinforcement\_learning/policy\_gradient.py. Check that it is correct by calling the train() function.

In [ ]:
from lxmls.reinforcement_learning.policy_gradient import train
train()

# Extra: Actor Critic for the CartPole task
Implement actor crtitic for the cartpole task by coding the critic forward pass in lxmls/reinforcement\_learning/policy\_gradient.py. Check it is correct by calling the train() function.

In [ ]:
from lxmls.reinforcement_learning.actor_critic import train
train()

# Exercise 6.6: Policy RNN for Part-of-Speech Tagging



As a last exercise, apply what you have learned to the RNN model seen in previous days. Implement REINFORCE to replace the maximum likelihood loss used on the RNN day. For this you can modify the PolicyRNN class in lxmls/deep learning/pytorch\_models/rnn.py
You can test your implementation by running the file lxmls/reinforcement_learning/RNN_Reinforce.py.